# Create the features from Topic Modeling
* using by_postal_code dataset

# Import

In [1]:
import pandas as pd
import json
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import pickle
import math 
from sklearn.model_selection import train_test_split

from scipy import sparse as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

In [2]:
RANDOM_STATE = 24

In [3]:
# 導入資料
train_df = pd.read_pickle('/home/adam/Steph_C/my_thesis/data/Train_by_postoal_code_without_review_pointwise_v3_3.pkl')
test_df = pd.read_pickle('/home/adam/Steph_C/my_thesis/data/Test_by_postoal_code_without_review_pointwise_v3_3.pkl')
all_df = pd.read_pickle('../Data/yelp/restaurant_only.pkl')

In [5]:
# chang column name
train_df.rename(columns = {'complementary':'poi_complementary'}, inplace = True)
test_df.rename(columns = {'complementary':'poi_complementary'}, inplace = True)

In [6]:
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
all_df = all_df.reset_index(drop=True)

In [7]:
print(train_df.shape , test_df.shape , all_df.shape)

(5984, 27) (3160, 27) (2112553, 20)


# Create Tokens

In [21]:
import spacy
import pyLDAvis.gensim_models
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel
from gensim import matutils

from nltk.stem import PorterStemmer
import nltk
from nltk.corpus import stopwords

In [ ]:
# EXAMPLE 
#import spacy
# from spacy.lang.en.examples import sentences 

# nlp = spacy.load("en_core_web_md")
# doc = nlp(sentences[0])
# print(doc.text)
# for token in doc:
#     print(token.text, token.pos_, token.dep_)

In [9]:
# # Our spaCy model:
# nlp = spacy.load("en_core_web_md")
# # Tags I want to remove from the text
# removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']
# tokens = []

# for summary in nlp.pipe(reports['summary']):
#     proj_tok = [token.lemma_.lower() for token in summary \
#                 if token.pos_ not in removal and not token.is_stop and token.is_alpha]
#     tokens.append(proj_tok)


In [10]:
# stop_words = stopwords.words('english')
nlp = spacy.load("en_core_web_md")
removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']

def preprocess(text : str) -> list:
    
    for summary in nlp.pipe([text]):
        proj_tok = [token.lemma_.lower() for token in summary \
                    if token.pos_ not in removal and not token.is_stop and token.is_alpha]

    return proj_tok

In [13]:
def get_attribute_tokens(df):
    
    attribute_list = ['Ambience' , 'GoodForMeal' ]
    att_list = []
    df = df.reset_index(drop= True)
    
    for i in range(len(df)):
        try:
            for k,v in df.attributes[i].items():
                if k in attribute_list:
                    if isinstance(v,dict):
                        for k_1 , v_1 in v.items():
                            if v_1:
                                att_list.append(k_1)
        except:
            pass

    return att_list

In [14]:
# Create tokens
def get_tokens(df):
    
    df['res_tokens'] = ''
    df['loc_tokens'] = ''
    df['res_tokens_with_att'] = ''
    df['loc_tokens_with_att'] = ''
    
    for i in range(len(df)):
        
        if i%1000==0:
            print(f'Now progress .... {i}')
        
        res_str = ''
        loc_str = ''

        res = df[df.name==df.name[i]]
        res = res[res.postal_code != df.postal_code[i]]

        loc = df[df.postal_code==df.postal_code[i]]
        loc = loc[loc.name != df.name[i]]

        # get res_string
        for j in Counter(res.text):
            removed = []
            if j not in removed:
                res_str+=j+' '
                removed.append(j)

        # get loc_string
        for j in Counter(loc.text):
            removed = []
            if j not in removed:
                loc_str+=j+' '
                removed.append(j)
        
        # get attributes tokens
        res_att = get_attribute_tokens(res)
        loc_att = get_attribute_tokens(loc)
        
        orig_list_res = preprocess(res_str)
        orig_list_loc = preprocess(loc_str)
        
        df['res_tokens'][i] = orig_list_res
        df['loc_tokens'][i] = orig_list_loc
        
        df['res_tokens_with_att'][i] = orig_list_res+res_att
        df['loc_tokens_with_att'][i] = orig_list_loc+loc_att
        
        
        
    
    print(f'Finish building .....')
    

    return df
    

In [15]:
# Takes a LONG time
train_df = get_tokens(train_df)
test_df = get_tokens(test_df)

Now progress .... 0


/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarn

Now progress .... 1000
Now progress .... 2000
Now progress .... 3000
Now progress .... 4000
Now progress .... 5000
Finish building .....
Now progress .... 0
Now progress .... 1000
Now progress .... 2000
Now progress .... 3000
Finish building .....


# LDA

In [27]:
def get_LDA(train_df , test_df ,tokens):
    
    train_df['LDA_'+tokens] = ''
    test_df['LDA_'+tokens] = ''
    dictionary = Dictionary(train_df[tokens])
    dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)
    
    #### doc2Bow
    train_corpus = [dictionary.doc2bow(doc) for doc in train_df[tokens]]
    test_corpus = [dictionary.doc2bow(doc) for doc in test_df[tokens]]

    # building models
    lda_model = LdaMulticore(corpus=train_corpus, id2word=dictionary, iterations=50, \
                             num_topics=10, workers = 4, passes=10)
    
    for i in range(len(train_corpus)):
        
        if i%1000==0:
            print(f'Now progress .... {i}')
        
        train_df['LDA_'+tokens][i] = lda_model[train_corpus][i]

    print(f'Finish building train .....')
    
    for i in range(len(test_corpus)):
        
        if i%1000==0:
            print(f'Now progress .... {i}')
        
        test_df['LDA_'+tokens][i] = lda_model[test_corpus][i]

    print(f'Finish building test .....')
    return train_df , test_df , lda_model

In [91]:
def get_LDA_tfidf(train_df , test_df,tokens ):
    
    train_df['LDA_'+tokens+'_tfidf'] = ''
    test_df['LDA_'+tokens+'_tfidf'] = ''

    vectorizer = TfidfVectorizer(min_df=10, max_df=0.2, max_features=1000)
    
    #### TF-IDF
    train_documents = [' '.join(tokens) for tokens in list(train_df[tokens])]
    test_documents = [' '.join(tokens) for tokens in list(test_df[tokens])]

    train_corpus = matutils.Sparse2Corpus(vectorizer.fit_transform(train_documents).T)
    test_corpus = matutils.Sparse2Corpus(vectorizer.fit_transform(test_documents).T)
    
    
    # building models
    id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())
    lda_model = LdaMulticore(corpus=train_corpus, id2word=id2word, iterations=50, \
                             num_topics=10, workers = 4, passes=10)
    
    for i in range(len(train_corpus)):
        
        if i%1000==0:
            print(f'Now progress .... {i}')
        
        train_df['LDA_'+tokens+'_tfidf'][i] = lda_model[train_corpus][i]

    print(f'Finish building train .....')
    
    for i in range(len(test_corpus)):
        
        if i%1000==0:
            print(f'Now progress .... {i}')
        
        test_df['LDA_'+tokens+'_tfidf'][i] = lda_model[test_corpus][i]

    print(f'Finish building test .....')
    return train_df , test_df , lda_model

In [92]:
train_df ,test_df , res_lda = get_LDA(train_df ,test_df , 'res_tokens')
train_df ,test_df , res_lda_w_attr = get_LDA(train_df ,test_df , 'res_tokens_with_att')
train_df ,test_df , loc_lda = get_LDA(train_df ,test_df , 'loc_tokens')
train_df ,test_df , loc_lda_w_attr = get_LDA(train_df ,test_df , 'loc_tokens_with_att')

Process ForkPoolWorker-164:
Process ForkPoolWorker-163:
Process ForkPoolWorker-161:
Process ForkPoolWorker-162:


KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "

In [93]:
train_df ,test_df , res_lda_tfidf = get_LDA_tfidf(train_df ,test_df , 'res_tokens_with_att')
train_df ,test_df , loc_lda_tfidf = get_LDA_tfidf(train_df ,test_df , 'loc_tokens_with_att')

Now progress .... 0


/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Now progress .... 1000
Now progress .... 2000
Now progress .... 3000
Now progress .... 4000
Now progress .... 5000
Finish building train .....
Now progress .... 0


/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Now progress .... 1000
Now progress .... 2000
Now progress .... 3000
Finish building test .....
Now progress .... 0


/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Now progress .... 1000
Now progress .... 2000
Now progress .... 3000
Now progress .... 4000
Now progress .... 5000
Finish building train .....
Now progress .... 0


/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Now progress .... 1000
Now progress .... 2000
Now progress .... 3000
Finish building test .....


In [94]:
res_lda_tfidf.print_topics(-1)

[(0,
  '0.018*"plan" + 0.006*"enjoy" + 0.006*"dave" + 0.006*"sorry" + 0.006*"inch" + 0.006*"saturday" + 0.005*"veggie" + 0.005*"smoothie" + 0.005*"perfection" + 0.005*"job"'),
 (1,
  '0.014*"card" + 0.009*"carmel" + 0.008*"noodle" + 0.007*"sample" + 0.007*"ham" + 0.006*"min" + 0.006*"coffee" + 0.005*"overall" + 0.005*"mac" + 0.005*"taco"'),
 (2,
  '0.029*"plan" + 0.007*"saturday" + 0.006*"calzone" + 0.006*"boil" + 0.006*"bun" + 0.006*"original" + 0.006*"setup" + 0.006*"certain" + 0.005*"high" + 0.005*"bell"'),
 (3,
  '0.018*"taco" + 0.010*"right" + 0.009*"head" + 0.009*"believe" + 0.007*"card" + 0.007*"plain" + 0.006*"pub" + 0.006*"flag" + 0.006*"positive" + 0.006*"question"'),
 (4,
  '0.009*"positive" + 0.008*"min" + 0.007*"carmel" + 0.007*"set" + 0.006*"shut" + 0.006*"tonight" + 0.006*"taco" + 0.006*"plan" + 0.006*"lemonade" + 0.005*"garage"'),
 (5,
  '0.014*"will" + 0.011*"plan" + 0.011*"bun" + 0.010*"bathroom" + 0.008*"example" + 0.006*"version" + 0.006*"market" + 0.006*"crazy" + 0

In [95]:
loc_lda_tfidf.print_topics(-1)

[(0,
  '0.027*"popular" + 0.027*"tempeh" + 0.019*"earn" + 0.019*"chorizo" + 0.019*"limit" + 0.019*"whiskey" + 0.019*"trimming" + 0.019*"mojos" + 0.018*"shade" + 0.018*"catch"'),
 (1,
  '0.022*"bj" + 0.018*"flour" + 0.017*"imho" + 0.017*"bowls" + 0.016*"eatery" + 0.016*"baked" + 0.016*"super" + 0.014*"photo" + 0.013*"chile" + 0.013*"value"'),
 (2,
  '0.015*"gallo" + 0.014*"heirloom" + 0.013*"skeptical" + 0.008*"bf" + 0.008*"opportunity" + 0.008*"robbie" + 0.008*"anytime" + 0.008*"ladt" + 0.007*"peeve" + 0.007*"single"'),
 (3,
  '0.029*"satisfied" + 0.022*"golden" + 0.019*"sweeten" + 0.018*"brew" + 0.017*"celiac" + 0.017*"spacious" + 0.014*"naan" + 0.013*"santucci" + 0.013*"current" + 0.013*"raspberry"'),
 (4,
  '0.049*"samosa" + 0.049*"wear" + 0.046*"dark" + 0.025*"route" + 0.025*"credit" + 0.021*"ny" + 0.019*"ball" + 0.019*"movie" + 0.016*"focus" + 0.016*"cater"'),
 (5,
  '0.022*"flatbread" + 0.021*"gnocchi" + 0.017*"elevate" + 0.015*"favs" + 0.014*"pound" + 0.014*"company" + 0.014*"bd

## Cal Cosine Similarity 

In [ ]:
# from gensim.matutils import cossim
# # Return cosine similarity between two sparse vectors. 
# # The similarity is a number between <-1.0, 1.0>, higher is more similar.
# print(cossim(train_df.LDA_res_tokens[1], train_df.LDA_loc_tokens[1]))

In [100]:
train_df.columns

Index(['business_id', 'stars_x', 'useful', 'funny', 'cool', 'text', 'date',
       'name', 'address', 'city', 'state', 'postal_code', 'latitude',
       'longitude', 'stars_y', 'review_count', 'is_open', 'attributes',
       'categories', 'hours', 'relevance', 'density', 'entropy', 'area_pop',
       'accessibility', 'poi_complementary', 'competitiveness', 'res_tokens',
       'loc_tokens', 'res_tokens_with_att', 'loc_tokens_with_att',
       'LDA_res_tokens', 'LDA_res_tokens_with_att', 'LDA_loc_tokens',
       'LDA_loc_tokens_with_att', 'LDA_res_tokens_with_att_tfidf',
       'LDA_loc_tokens_with_att_tfidf', 'cosine_sim', 'cosine_sim_w_attr',
       'cosine_sim_tfidf'],
      dtype='object')

In [101]:
# create Affinity and Complementary
from gensim.matutils import cossim

def get_cos_sim(df):
    
    df['cosine_sim'] =''
    df['cosine_sim_w_attr'] =''
    df['cosine_sim_tfidf'] =''
    
    for i in range(len(df)):
        df['cosine_sim'][i] = cossim(df.LDA_res_tokens[i], df.LDA_loc_tokens[i])
        df['cosine_sim_w_attr'][i] = cossim(df.LDA_res_tokens_with_att[i], df.LDA_loc_tokens_with_att[i])
        df['cosine_sim_tfidf'][i] = cossim(df.LDA_res_tokens_with_att_tfidf[i], df.LDA_loc_tokens_with_att_tfidf[i])
    
    return df

In [102]:
train_df = get_cos_sim(train_df)
test_df = get_cos_sim(test_df)

/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys

## Affinity & Complementary

In [103]:
def get_aff_comp(df) : 
    
    df['affinity'] = ''
    df['complementary'] = ''
    df['affinity_with_att'] = ''
    df['complementary_with_att'] = ''
    df['affinity_tfidf'] = ''
    df['complementary_tfidf'] = ''
    
    for i in range(len(df)):
        affinity = 0
        complementary = 0
        affinity_with_att = 0
        complementary_with_att = 0
        affinity_tfidf = 0
        complementary_tfidf = 0
        
        for j in df.LDA_res_tokens[i]:
            for k in df.LDA_loc_tokens[i]: 
                if j[0] in k:
                    affinity += j[1]*k[1]
                    complementary += j[1]*(1-k[1])
        
        
        for j in df.LDA_res_tokens_with_att[i]:
            for k in df.LDA_loc_tokens_with_att[i]: 
                if j[0] in k:
                    affinity_with_att += j[1]*k[1]
                    complementary_with_att += j[1]*(1-k[1]) 
        
        for j in df.LDA_res_tokens_with_att_tfidf[i]:
            for k in df.LDA_loc_tokens_with_att_tfidf[i]: 
                if j[0] in k:
                    affinity_tfidf += j[1]*k[1]
                    complementary_tfidf += j[1]*(1-k[1]) 
        
        df['affinity'][i] = affinity
        df['complementary'][i] = complementary
        df['affinity_with_att'][i] = affinity_with_att
        df['complementary_with_att'][i] = complementary_with_att
        df['affinity_tfidf'][i] = affinity_tfidf
        df['complementary_tfidf'][i] = complementary_tfidf
        
    return df

        

In [104]:
train_df = get_aff_comp(train_df)
test_df = get_aff_comp(test_df)

/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarn

In [105]:
for i in ['affinity','complementary','affinity_with_att','complementary_with_att','cosine_sim','affinity_tfidf','complementary_tfidf']:
    train_df[i] = train_df[i].astype('float')
    test_df[i] = test_df[i].astype('float')

In [106]:
train_df.to_pickle('/home/adam/Steph_C/my_thesis/data/Train_by_postoal_code_pointwise_v3_3_tfidf.pkl')
test_df.to_pickle('/home/adam/Steph_C/my_thesis/data/Test_by_postoal_code_pointwise_v3_3_tfidf.pkl')